# TP 4 - Un peu de cryptanalyse

In [4]:
from OutilsCrypto import *

Nous allons étudier dans ce TP quelques attaques possibles contre les algorithmes symmétriques étudiés lors des TP 2 et 3.

## 1. Attaque par dictionnaire

Une première idée est d'utiliser la force brute pour déchiffrer le massage avec toutes les clés possibles. Si le calcul n'est en général pas si long avec les algorithmes que nous avons étudiés, il reste le problème de détecter le bon message parmi le grand nombre de sorties obtenues.

La solution que nous allons étudier ici est de détecter dans chaque tentative de déchiffrement la probabilité que le message obtenu soit du texte en langue française. Pour celà, nous allons créer un arbre modélisant l'ensemble des mots d'un dictionnaire, puis nous allons calculer <b>l'incidence</b> d'une phrase comme le nombre de mots du dictionnaire qu'elle contient en parcourant cet arbre.

Par exemple, la chaîne de caractère "cetexteestlisible" contient les mots "ce" "texte" "est" "lisible" mais aussi d'autres mots comme "cet" "es" "ete" etc. Une chaîne qui n'aurait aucun sens comme "zklfhibfealoejdf" contiendra a priori moins de mots du dictionnaire. Ce sera particulièrement vrai sur des textes plus longs.

In [5]:
#Création d'un arbre représentant le dictionnaire

def MonDico() :

    Racine=dict()

    def constructBranche(mot):
        Arbre = Racine;
        for c in mot:
            #Si le cara c n'existe pas on le crée (permet en plus de gérer les répétitions éventuelles)
            if not (c in Arbre):
                Arbre[c] = dict()
                Arbre[c]['FINMOT']=False
            #On avance dans l'arbre
            Arbre = Arbre[c]
        #Arrivé à la fin on marque que le mot est fini
        Arbre["FINMOT"] = True

    dico = "Dictionnaires/DictionnaireFR.txt"

    with open(dico, "r", encoding='utf8') as f :
        for ligne in f.readlines() :
            constructBranche(Filtre(ligne.strip()))
    return Racine

#Calcul la pertinence d'une phrase avec l'arbre
def pertinence(phrase, arbre) :
    pert=0
    n=len(phrase)
    for i in range(n) :
        test=True
        positionDico=arbre
        for j in range(i, n) :
            cara=phrase[j]
            try : x=positionDico[cara]
            except : break
            if(x['FINMOT']) :
                pert+=1
            positionDico=x
    return pert

Afin de tester ces algorithmes, commencez par charger le dictionnaire que nous allons utiliser en exéutant le bloc suivant. Si vous ne l'avez pas fait lors du TP4, vous prendrez le soin auparavant de créer sur votre espace personnel sur le serveur Salazar un répertoire nommé <b>Dictionnaires</b> et d'y placer le fichier <b>DictionnaireFR.txt</b> que vous récupérerez sur UMTICE et qui contient 336531 mots de la langue française.

In [6]:
DICO_FR = MonDico()

Nous pouvons désormais calculer la pertinence d'une phrase dans ce dictionnaire comme dans le bloc suivant.

In [7]:
pertinence("jevoussouhaiteunebonneanneetousmesvoeuxdereussite",DICO_FR)

52

<b>Exercice 1.</b> Utiliser cette fonction pour écrire la fonction <code>attaqueCesar(message)</code> qui décrypte <code>message</code> par force brute et retourne le déchiffrement qui a la meilleure <code>pertinence</code>. Vous afficherez le nombre de mots testés ainsi que le temps mis par l'algorithme.

In [8]:
from datetime import datetime


def attaqueCesar(message):
    t0 = datetime.now()  # l'heure à l'instant présent
    n = 0  # pour compter le nombre de clés testées
    meilleur = 0  # meilleure pertinence
    meilleurTexte = ""
    meilleurCle = 0

    for cle in range(26):
        n += 1
        texte = dechiffreCesar(message, cle)

        if pertinence(texte, DICO_FR) > meilleur:
            meilleur = pertinence(texte, DICO_FR)
            meilleurTexte = texte
            meilleurCle = cle

    print(
        "{} clé(s) ont étés testées en {} seconde(s),".format(
            n, (datetime.now() - t0).total_seconds()
        )
    )
    return "Le meilleur message est : {} avec la clé {} et une pertinence de {}".format(
        meilleurTexte, meilleurCle, meilleur
    )

Tester votre algorithme en exécutant le bloc suivant.

In [9]:
print(attaqueCesar("PRFGDHNAQZRZRCNFGREEVOYRYRPUVSSERZRAGQRPRFNE"))

26 clé(s) ont étés testées en 0.002183 seconde(s),
Le meilleur message est : cestquandmemepasterriblelechiffrementdecesar avec la clé 13 et une pertinence de 44


<b>Exercice 2.</b> De la même façon, écrire la fonction <code>attaqueAffine(message)</code> qui décrypte <code>message</code> par force brute et retourne le déchiffrement qui a la meilleure <code>pertinence</code>. Vous afficherez le nombre de mots testés ainsi que le temps mis par l'algorithme.

In [10]:
def attaqueAffine(message):
    t0 = datetime.now()	# l'heure à l'instant présent
    n = 0 # pour compter le nombre de clés testées
    meilleur = 0 # meilleure pertinence
    meilleurTexte = ""
    meilleurCle = 0
    
    for a in range(26):
        for b in range(26):
            n += 1
            texte = dechiffreAffine(message, a, b)
            
            if pertinence(texte, DICO_FR) > meilleur:
                meilleur = pertinence(texte, DICO_FR)
                meilleurTexte = texte
                meilleurCle = (a, b)
        

    print("{} clé(s) ont étés testées en {} seconde(s),".format(n, (datetime.now()-t0).total_seconds()))
    
    return "Le meilleur message est : {} avec la clé {} et une pertinence de {}".format(meilleurTexte, meilleurCle, meilleur)


In [11]:
attaqueAffine("ntjmahjkthpnxparftpyotncfwwgtrtumhwwfut")

676 clé(s) ont étés testées en 0.043778 seconde(s),


'Le meilleur message est : cestpasbeaucoupmieuxlechiffrementaffine avec la clé (3, 7) et une pertinence de 44'

## 2. Indice de coincidence

### 2.1 Un problème de taille...

Attaquons nous maintenant au chiffrement de Vigenère. Si la clé est de taille $k$, il y aura $26^k$ clés à tester, et comme on ne connaît pas la taille $k$, il faut tester $26+26^2+26^3+\dots$ clés, ce qui commence à faire beaucoup !

De plus, si la taille de la clé est proche de la taille du message, on se rapproche d'un chiffrement de Vernam pour lequel il y a de nombreux déchiffrements dont la pertinence sera élevée, y compris des mauvais déchiffrements !

<b>Exercice 3.</b> Écrire la fonction <code>attaqueVigenere(message)</code> qui décrypte le texte <code>message</code> par force brute en essayant toutes les clés de taille inférieure ou égale à la taille de <code>message</code>. Vous afficherez le nombre de mots testés ainsi que le temps mis par l'algorithme.

In [12]:
# Aide : le code suivant permet d'afficher tous les mots de longueur k sur l'alphabet a, b, ..., z
import string
import itertools

alphabet = list('abcdefghijklmnopqrstuvwxyz')
k = 2
for word in itertools.product(alphabet, repeat=k):
    word = (''.join(word))
    print(word)


aa
ab
ac
ad
ae
af
ag
ah
ai
aj
ak
al
am
an
ao
ap
aq
ar
as
at
au
av
aw
ax
ay
az
ba
bb
bc
bd
be
bf
bg
bh
bi
bj
bk
bl
bm
bn
bo
bp
bq
br
bs
bt
bu
bv
bw
bx
by
bz
ca
cb
cc
cd
ce
cf
cg
ch
ci
cj
ck
cl
cm
cn
co
cp
cq
cr
cs
ct
cu
cv
cw
cx
cy
cz
da
db
dc
dd
de
df
dg
dh
di
dj
dk
dl
dm
dn
do
dp
dq
dr
ds
dt
du
dv
dw
dx
dy
dz
ea
eb
ec
ed
ee
ef
eg
eh
ei
ej
ek
el
em
en
eo
ep
eq
er
es
et
eu
ev
ew
ex
ey
ez
fa
fb
fc
fd
fe
ff
fg
fh
fi
fj
fk
fl
fm
fn
fo
fp
fq
fr
fs
ft
fu
fv
fw
fx
fy
fz
ga
gb
gc
gd
ge
gf
gg
gh
gi
gj
gk
gl
gm
gn
go
gp
gq
gr
gs
gt
gu
gv
gw
gx
gy
gz
ha
hb
hc
hd
he
hf
hg
hh
hi
hj
hk
hl
hm
hn
ho
hp
hq
hr
hs
ht
hu
hv
hw
hx
hy
hz
ia
ib
ic
id
ie
if
ig
ih
ii
ij
ik
il
im
in
io
ip
iq
ir
is
it
iu
iv
iw
ix
iy
iz
ja
jb
jc
jd
je
jf
jg
jh
ji
jj
jk
jl
jm
jn
jo
jp
jq
jr
js
jt
ju
jv
jw
jx
jy
jz
ka
kb
kc
kd
ke
kf
kg
kh
ki
kj
kk
kl
km
kn
ko
kp
kq
kr
ks
kt
ku
kv
kw
kx
ky
kz
la
lb
lc
ld
le
lf
lg
lh
li
lj
lk
ll
lm
ln
lo
lp
lq
lr
ls
lt
lu
lv
lw
lx
ly
lz
ma
mb
mc
md
me
mf
mg
mh
mi
mj
mk
ml
mm
mn
mo
mp
mq
mr
ms
mt
mu
m

In [13]:
def attaqueVigenere(message):
    t0 = datetime.now()	# l'heure à l'instant présent
    n = 0

    meilleur = 0 # meilleure pertinence
    meilleurTexte = ""
    meilleurCle = ""
    
    for k in range(1, 10):
        
        for cle in itertools.product(alphabet, repeat=k):
            n += 1
            texte = dechiffreVigenere(message, cle)
            
            if pertinence(texte, DICO_FR) > meilleur:
                meilleur = pertinence(texte, DICO_FR)
                meilleurTexte = texte
                meilleurCle = cle

    print("{} mot(s) ont étés testés en {} seconde(s),".format(n, (datetime.now()-t0).total_seconds()))
    
    return "{} avec la clé {} et une pertinence de {}".format(meilleurTexte, meilleurCle, meilleur)


Tester votre fonction en exécutant le bloc suivant.

In [ ]:
texte_chiffre = chiffreVigenere("test", "cle")
print(texte_chiffre)
print("Le chiffrement de Vigenere du message 'test' avec la clé 'cle' est 'vpwv'")
texte_decrypte = attaqueVigenere("vpwv")
print("Le résultat de l'attaque est : ", texte_decrypte)

Que constatez-vous ? Expliquer pourquoi ça ne fonctionne pas.

<div style="background-color:rgba(0, 255, 0, 0.19);padding:3%;">
<b>Réponse : Le programme ne se termine jamais.</b>Cela peut être causé par : 
<ul><li>
Sensibilité à la longueur de la clé : L'algorithme d'attaque utilisé (force brute) peut ne pas être efficace pour des clés de longueur variable. Il peut avoir du mal à converger vers la clé correcte, surtout si la longueur de la clé est grande.</li>
<li>
Dépendance de la fonction de pertinence : La fonction de pertinence utilisée dans l'attaque (basée sur les occurrences de mots français) peut ne pas être suffisamment discriminante pour identifier la clé correcte.</li>
</ul>
</div>

Bien entendu, on peut limiter la taille des clés à tester. Écrire la fonction <code>attaqueVigenere(message, maxSize)</code> qui décrypte le texte <code>message</code> par force brute en essayant toutes les clés de taille inférieure ou égale à <code>maxSize</code>.

In [16]:
import string
import itertools

def attaqueVigenere(message,maxSize):
    t0 = datetime.now()	# l'heure à l'instant présent
    n = 0

    meilleur = 0 # meilleure pertinence
    meilleurTexte = ""
    meilleurCle = ""
    
    for k in range(1, 10):
        
        
        for cle in itertools.product(alphabet, repeat=k):
            n += 1
            texte = dechiffreVigenere(message, cle)
            if(n>=maxSize):
                    print("{} mot(s) ont étés testés en {} seconde(s),".format(n, (datetime.now()-t0).total_seconds()))
    
                    return "{} avec la clé {} et une pertinence de {}".format(meilleurTexte, meilleurCle, meilleur)
            
            if pertinence(texte, DICO_FR) > meilleur:
                meilleur = pertinence(texte, DICO_FR)
                meilleurTexte = texte
                meilleurCle = cle

    print("{} mot(s) ont étés testés en {} seconde(s),".format(n, (datetime.now()-t0).total_seconds()))
    
    return "{} avec la clé {} et une pertinence de {}".format(meilleurTexte, meilleurCle, meilleur)

Tester votre fonction en exécutant le bloc suivant.

In [17]:
attaqueVigenere('lzcgfbpzinlxqfwwyidzrppepyigpxwyidzrppwcyxg',3)

3 mot(s) ont étés testés en 0.000665 seconde(s),


"lzcgfbpzinlxqfwwyidzrppepyigpxwyidzrppwcyxg avec la clé ('a',) et une pertinence de 7"

### 2.2 Trouver la taille de la clé

L'<b>indice de coïncidence</b> est une technique de cryptanalyse inventée par William F. Friedman en 1920 et améliorée par  Solomon Kullback.

L'indice permet de savoir si un texte a été chiffré avec un chiffrement mono-alphabétique (de type César ou affine) ou un chiffrement poly-alphabétique (comme Vigenère) en étudiant la probabilité de répétition des lettres du message chiffré. Il donne également une indication sur la longueur de la clé probable.

L'indice se calcule avec la formule suivante :
$$
IC = \sum _{{q=A}}^{{q=Z}}{\frac  {n_{{q}}(n_{{q}}-1)}{n(n-1)}}
$$
avec $n$ le nombre de lettres total du message, $n_{A}$ le nombre de $A$, $n_{B}$ le nombre de $B$, etc.

En français, l'indice de coïncidence vaut environ 0,0746. Dans le cas de lettres uniformément distribuées (contenu aléatoire sans biais), l'indice se monte à 0,0385. L'indice ne varie pas si une substitution monoalphabétique des lettres a été opérée au préalable, c’est-à-dire que si l'on remplace par exemple 'a' par 'z' et 'z' par 'a', l'indice ne changera pas.

<b>Exercice 4.</b> Écrire la fonction <code>ic(texte)</code> qui calcule l'indice de coïncidence de <code>texte</code>.

In [18]:
def ic(texte):
    n = len(texte)
    alphabet = list(string.ascii_lowercase)
    somme = 0
    for lettre in alphabet:
        somme += texte.count(lettre)*(texte.count(lettre)-1)
    return somme/(n*(n-1))

Tester votre fonction en exécutant le bloc suivant.

In [19]:
texte = "Quarante-deux ! cria Loonquawl. Et c est tout ce que t'as à nous montrer au bout de sept millions et demi d'années de boulot ? J'ai vérifié très soigneusement, dit l'ordinateur, et c'est incontestablement la réponse exacte. Je crois que le problème, pour être tout à fait franc avec vous, est que vous n'avez jamais vraiment bien saisi la question."
texte = Filtre(texte)
print(ic(texte))
print(ic(chiffreCesar(texte,3)))
print(ic(chiffreAffine(texte,3,7)))
print(ic(chiffreVigenere(texte,"cle")))

0.07406808877397113
0.07406808877397113
0.07406808877397113
0.04589528118939883


Analysez les résultats obtenus.

<div style="background-color:rgba(0, 255, 0, 0.19);padding:3%;">
<b>Réponse :</b>
<ul>
<li>
    Texte Original :
    <ul>
        <li>Indice de coïncidence : ~0.0741 (proche de l'indice pour le français).</li>
        </ul>
</li>
<li>
    Chiffrement César (décalage de 3) :
    <ul>
        <li>Indice de coïncidence : ~0.0741 (proche de l'indice pour le français).</li>
        <li>Les chiffrements de César avec un décalage constant ne modifient pas la distribution des lettres dans le texte de manière significative, d'où un indice similaire à celui du texte original.</li>
        </ul>
</li>
<li>
    Chiffrement Affine (a=3, b=7) :
    <ul>
        <li>Indice de coïncidence : ~0.0741 (proche de l'indice pour le français).</li>
        <li>Les chiffrements affines, bien que plus complexes que les Césars, ne modifient pas radicalement la distribution des lettres, ce qui se reflète dans des indices similaires.</li>
        </ul>
</li>
<li>
    Chiffrement Vigenère (clé "cle") :
    <ul>
        <li>Indice de coïncidence : ~0.0459.</li>
        <li>Le chiffrement de Vigenère, étant polyalphabétique, a tendance à réduire l'indice de coïncidence par rapport aux chiffrements mono-alphabétiques. Cela est dû à la variabilité induite par l'utilisation d'une clé.</li>
        </ul>
</li>
</ul>
</div>

Regardons désormais comment l'indice de coïncidence peut nous donner des indices sur la taille de la clé en cas de chiffrement par la méthode de Vigenère.

In [20]:
texte_chiffre_vigenere = chiffreVigenere(texte,"cle")

print([ic(texte_chiffre_vigenere[i::1]) for i in range(1)])
print([ic(texte_chiffre_vigenere[i::2]) for i in range(2)])
print([ic(texte_chiffre_vigenere[i::3]) for i in range(3)])
print([ic(texte_chiffre_vigenere[i::4]) for i in range(4)])
print([ic(texte_chiffre_vigenere[i::5]) for i in range(5)])

[0.04589528118939883]
[0.043795620437956206, 0.044880174291938996]
[0.06788766788766788, 0.07814407814407814, 0.06788766788766788]
[0.04731457800511509, 0.03994732221246708, 0.03950834064969271, 0.04477611940298507]
[0.0430976430976431, 0.052525252525252523, 0.041750841750841754, 0.03284416491963662, 0.045422781271837874]


Comment peut-on déduire de ces calculs la taille de la clé ?

<div style="background-color:rgba(0, 255, 0, 0.19);padding:3%;">
<b>Réponse : </b>En analysant les variations de l'indice de coïncidence pour différentes longueurs de clé, on peut repérer la longueur de clé probable en identifiant les pics dans les valeurs de l'indice de coïncidence. Dans cet exemple, les pics à la deuxième, troisième, quatrième et cinquième séquence indiquent des longueurs potentielles de clé. Il serait judicieux de vérifier plus en détail autour de ces valeurs pour confirmer la longueur de la clé.
</div>

## 3. Attaque fréquentielle

Une fois qu'on connaît la taille de la clé, il pourrait être envisageable de décrypter avec une attaque par force brute, mais il est aussi possible de faire une attaque fréquentielle.

En effet, le calcul de l'indice de coïncidence, proche de $0,075$, nous indique la longueur de la clé, mais aussi que le texte semble bien rédigé en français. Si le texte est assez long et la clé de taille raisonnable, l'idée est de considérer séparément les chaînes de caractères obtenues en prenant les lettres qui ont subi le même décalage, puis de chercher la taille de ce décalage en cherchant la lettre la plus fréquente et en supposant que cette lettre est obtenue en chiffrant la lattre $E$ (la plus fréquente dans la langue française).

<b>Exercice 5.</b> Écrire la fonction <code>freq(txt)</code> qui calcule la fréquence des lettres de l'alphabet a..z dans le texte <code>txt</code>. Elle doit renvoyer un tableau contenant ces fréquences.

In [21]:
def freq(txt):
    alphabet = "abcdefghijklmnopqrstuvwxyz"
    freq_table = [0] * 26  # Initialiser le tableau des fréquences

    for char in txt:
        if char in alphabet:
            index = alphabet.index(char)
            freq_table[index] += 1


    return freq_table

Tester votre fonction en exécutant le bloc suivant.

In [22]:
freq("alphabet") # doit renvoyer [2, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]

[2, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]

Écrire la fonction <code>maxFreq(txt)</code> qui renvoie la position de la lettre la plus fréquente dans le texte <code>txt</code> (0 pour a, 1 pour b, etc.)
S'il y a plusieurs lettres ayant cette fréquence d'apparition, la fonction en renvoie une seule.

In [23]:
def maxFreq(txt):
    freq_table = freq(txt)
    max_freq = max(freq_table)
    index = freq_table.index(max_freq)
    return index

Tester votre fonction en exécutant le bloc suivant.

In [24]:
maxFreq("ephemeride") # doit renvoyer 4 car la lettre e est la plus fréquente et elle en position 4 dans l'alphabet

4

Écrire la fonction <code>attaqueFreqVigenere(txt,lg)</code> qui décrypte le texte <code>txt</code> chiffré par le chiffrement de Vigenère avec une clé de longueur <code>lg</code>.

In [70]:

def attaqueFreqVigenere(txt, lg):
    t0 = datetime.now()
    n = 0
    # n is the number of words tested
    
    
    # On calcule les fréquences de chaque sous-texte
    freq_table = []
    for i in range(lg):
        freq_table.append(freq(txt[i::lg]))
        
    # On calcule les indices de coincidence de chaque sous-texte
    ic_table = []
    for i in range(lg):
        ic_table.append(ic(txt[i::lg]))
        
    # On calcule les décalages de chaque sous-texte
    
    decalage_table = []
    for i in range(lg):
        decalage_table.append(maxFreq(txt[i::lg]))
        
        
    # On calcule la clé
    cle = ""
    for i in range(lg):
        cle += chr(97 + (decalage_table[i] - 4) % 26)
    
    # On déchiffre le texte
    texte = dechiffreVigenere(txt, cle)
    

    print("L'algorythme a pris {} seconde(s),".format(lg, (datetime.now()-t0).total_seconds()))
    return "{} avec la clé partielle {}".format(texte, cle)

Tester votre fonction en exécutant le bloc suivant.

In [71]:
texte = "Quarante-deux ! cria Loonquawl. Et c'est tout ce que t'as à nous montrer au bout de sept millions et demi d'années de boulot ? J'ai vérifié très soigneusement, dit l'ordinateur, et c'est incontestablement la réponse exacte. Je crois que le problème, pour être tout à fait franc avec vous, est que vous n'avez jamais vraiment bien saisi la question."
texte = Filtre(texte)
texte_chiffre_vigenere = chiffreVigenere(texte,"bcd")
print(texte_chiffre_vigenere)
print(attaqueFreqVigenere(texte_chiffre_vigenere,3))

rwdscquggfwadtlbnrpptvczmgwdgvuvrvvffsxfvdtcqpwvnqquthscxcqxufhtgsuolmnlppvfvgfolecqoghtfhcqxmqwkclwgujhlfvufuvpkjogxtgpfpwekwmquekqbvhvthuehtvloerovhtvdcnhngqundsgsppvfgabewflhdtrjutvgofrupdofohqqxsgwsgwpwwbhdjviscqdcyfeypwvfuwrwhwqxtpdwgckcpbkvwtdjohovejgqtcltkobsxfuwjqq
3 mot(s) ont étés testés en 0.000783 seconde(s),
quarantedeuxcrialoonquawletcesttoutcequetasanousmontrerauboutdeseptmillionsetdemidanneesdeboulotjaiverifietressoigneusementditlordinateuretcestincontestablementlareponseexactejecroisqueleproblemepouretretoutafaitfrancavecvousestquevousnavezjamaisvraimentbiensaisilaquestion avec la clé partielle bcd


<b>Exercice 6.</b> Vérifions si vous avez compris ! J'ai chiffré un passage d'un livre à l'aide la méthode de Vigenère, avec une clé de longueur inférieure à 10. Voici ce que j'ai obtenu :
aalwvvarfwbxsjgvhschgaseqjzxgrwgmgzxpcwvqgohdlwjcipycjgrhoywsjzvocyvdcijpcffsjelqzppwvzwuzeusisefrlqgdceocpxftcdysfqsfbuqflifrwttwdvoehvvodswiozewyvhzbtfwghavbkpsdosssioslxosczdslyckfvecfuqvdcgglqqzsezsbxscsjazplzvhaqqzqhzblqsyfcisuqtzxzvfcqdlujzgjmqchrvjfffpwsddcqgzosebvxazlzvdcggqlrvzvpsgrgzbzfwpvwcmrhotwrljrsipgoegdabpvdiwkgbuhbvgrugbxczsgmwdfcdavpswdtlavqallgasjggquoeqyufchzzuzqidhavbkxsdgsxfveeflavbvzhlyckfvmiehzvhmaiddjvnttodvstsmawwhcsgtgfnradscqjpqhtvregposuoduscyclgrhskpwjocmdwdqvieqtcrwusldsifsjgzhsfqsgflpsyfstceecxpsvskgbpocxwhgstpdcotmpwhocozpsohjfhiqzllhwcifwqloehdabtqhvzcuupqqvgvehcddzrvysywrvjvxcassvskmdclgusjbfzscihzabdladseesddidwcusfgstskfsnooihvdoglgjoefsorbkjfggqdwksjbfpvsehrhsnsffrzsowlhvotqiitizjfggllavbkpofqgzbtqfpdafiimftwvdskuefhocuvnfpjsfavffthhiweuhpjfrbuucdhhiwrzuwhzlazzsfaqvzlueflbvjfgglsojqfzbfhgvgkgbtqgvbjqwwpsiwkqfllhcsgdsfysusjbzfvuioepgdxdcwtqgndfzzpmrfpsgfzeoghixzvpoyvgfbzzgzxqzoeostjbffrzhppozgtqzflelwmaidfcebruhpwjfijmdaustwvzsghikdcggclservepthbjrvxoehfisjqqzqhvbkqrpycjxfgwdvoeqvealjwhivesescihveicycjozxsdvcdpiqgyhrvgzdsaoijelqrpvscsmqfoxbmccxsrhfvbtabdwflwjmbexbvvvxwnhojqvzrlqhvjvdgwdjfikqgaksiwhgsohgtwvglwdhvfiqbpoizafzhchelsuqgtozlgzabdhhusjroywojarscclsjafdowhgdozejzxgfarfvppokwhgsdfceqzesdsoizvzqsdwesdqbeuwxcldsfarvjfedcrdfgzfwzqgksemqpvskzrocyvhrbtqrpycjzfugohtvfmaidiozhvezflfvoljmpxlvpcaitviefvrzpwdlwjeoywrvqvfhpysiwkqgfsfvavpcywcefvyoctivzvydchwehvpoyvzffudsohzlbzhscvavftudzxfcsjescywtsjubyrasfrnzpvelsmaidpomsqdsygijavdqtsclfcqgbxocwkqgpwfrbxqfpvrfbkhcfvomsqqbclqywdabtqhvzcuupqqvgrzggrijrrzgxdzlhkqqzqhisctcxpsaoldotvdvikqhchsksmmwyfijoeejzxgzzdmicdwktruhcricsipoyvzvgrnzphhvasdodvsizrbcfvgzsiqrpvsjdzqrdvoegmaiddjvqlzsruwwtvbsciwuszxofuozhcmpzxfvarovllfvhdqgzvarwjvsxhglwjfsyxglfdqgrdfusjocxpslbrfvwhhvsobsclavbkqjzxgdsuabydhvgcmtcrwusldeflglfxuhohjfgtabnhdkwfzgdxpcwdqgpasddkqgohdrgjucymsdseescywjdfgfchxvhvdoghqusumwyosjxfgwdvoeqvesaksdsiqgohafbtaicwjfmrsspwdfiidsyycpsipsxddffkqzpvcwtiqgdbagoktwbxsjarugeucddvggpvrvavegpppcosxsdyclgdqrzqbrhvezlsflrvzqprdzbzmhchelcepsnkwwtiqonkohivbodgoegmagxhhycuqglgazfrnzpvrvzrzowbgvrvxodbbkvvespwzrrvpinwwfbaqapqgvfmugariirvdcfwsizveffvsjdvdbtfwvijqgohafbvzbppwdcifswsclfcmhedelsimazqhfiimjpfoufvegphhgzfzupurrbjxsdywjqvdsdgscvfyapxbgczsblurrwxgeflfvgkqfldxrarugpqtfbtqrlqgjceoccsgtoiosdwiessxsdviisuabelzesjqfposmsimdlvjfijysorbeokqgwdzfuzciptizsjfqzpavzryspozvavysohjfgvzgpluesdqbevdcszzgohgruvegpdjvqjqgdbzccxugxhgucefzpoosmiubekstcdbzttivbvzsdwelsgxidfcddiqvpqgzpcqazqwehvxztjseqvesywwkgroqcrwkfvpioriszvesdiciqveofgotwvggpvocozpsohqvhrgltowrwiqhpufzpcqxpgstclhftvrrbjxvfpoewkqsyqoxsrzhghfjzveplvtfbuesyiotsuqzpfivwcpswdvrweqzlpstvrzqpwseczdspwvzrvggptizqiaialgjozfofpwcwvgrpvazojysdgscskqfpvsegrpatuoehcqbzppiwcxsausdwvdxpgstclhftvrrbjxsdwsessdsdgsjsjqbeuozzcqgnhjzqvzsqdgkscqalogldvdwpxfvbcgwlxpzsemjpfqvhkqocpsvagawdrbesvcipyclgdqdchhrhvexpiwjrveqpqrisuqgzqdzsuqgedztceehcxwkdrdzlootvvfsohzycdyswhqisrfsfuzlwdqaplzxfzzqlgsjrvzhdhhjisuhnhhkszzxfuszueaatqwvijqqluwcommwesclfrpjpugrwiqefhzhilzrpszlgwafetivzluallgascqzllgjsimwohqfhvocxpslbgmefhhuswuqpozvgrrwygosozegpuafbmazwhdvbjqicgsjqrdhpvtrwjmwexbvtfugnhhksiqtwhlzcecipuwvbuqgzowusemjllhvhvnoelglfmaidfskozfiyharbzqfplbxseusfvsuswmwchqfagdsygfvelqzpsfvazqfghblbvbcfyozhjgfwhqfigpsnrimfzdjzwfvjrxsfuwesjfwxdpcsvzsqiskelawohdcijecwlrvelqzpvhiczeefdzzhvedclbtwgmzpvrvxrzcxpsvghgwdhzvjvzhpqhiscmqphgtcdysfqstcldcyqslbzcipviizvycxhbkolsidwsusmahchoiqyuhpfhlfvocwrgjocqazqidsefefluioepwevoegtqgdhrvrvocfysihvcizwwuwvzbpvrrbjhcdpwesjpsolodoefsegsodcaflwwfbjeqthbkwwuefhguoeejzvgldvdppvrfarubpvcdoktsxdhzelqgddwehvedflgjwvljzxggoihceustcdyscfsgsibsexscqfzgzosizvdsdwsusdqguriiguqzlpstvrzqpwsusctcxpsvhuqztqxlgkuqpgixfrzrerik

Trouver la longueur de la clé que j'ai utilisée puis décrypter le passage à l'aide d'une attaque fréquentielle.
Inscrivez ensuite dans le bloc ci-dessous le nom de l'auteur et le titre du livre dont le passage est tiré.

In [72]:
# à compléter : recherche de la taille de la clé
def rechercheTaillCle(txt):
    # use ic to find the best key length
    ic_list = []
    for i in range(1, 20):
        ic_list.append(ic(txt[i::i]))
    return ic_list.index(max(ic_list)) + 1
    

In [74]:
# à compléter : attaque fréquentielle

txt = "aalwvvarfwbxsjgvhschgaseqjzxgrwgmgzxpcwvqgohdlwjcipycjgrhoywsjzvocyvdcijpcffsjelqzppwvzwuzeusisefrlqgdceocpxftcdysfqsfbuqflifrwttwdvoehvvodswiozewyvhzbtfwghavbkpsdosssioslxosczdslyckfvecfuqvdcgglqqzsezsbxscsjazplzvhaqqzqhzblqsyfcisuqtzxzvfcqdlujzgjmqchrvjfffpwsddcqgzosebvxazlzvdcggqlrvzvpsgrgzbzfwpvwcmrhotwrljrsipgoegdabpvdiwkgbuhbvgrugbxczsgmwdfcdavpswdtlavqallgasjggquoeqyufchzzuzqidhavbkxsdgsxfveeflavbvzhlyckfvmiehzvhmaiddjvnttodvstsmawwhcsgtgfnradscqjpqhtvregposuoduscyclgrhskpwjocmdwdqvieqtcrwusldsifsjgzhsfqsgflpsyfstceecxpsvskgbpocxwhgstpdcotmpwhocozpsohjfhiqzllhwcifwqloehdabtqhvzcuupqqvgvehcddzrvysywrvjvxcassvskmdclgusjbfzscihzabdladseesddidwcusfgstskfsnooihvdoglgjoefsorbkjfggqdwksjbfpvsehrhsnsffrzsowlhvotqiitizjfggllavbkpofqgzbtqfpdafiimftwvdskuefhocuvnfpjsfavffthhiweuhpjfrbuucdhhiwrzuwhzlazzsfaqvzlueflbvjfgglsojqfzbfhgvgkgbtqgvbjqwwpsiwkqfllhcsgdsfysusjbzfvuioepgdxdcwtqgndfzzpmrfpsgfzeoghixzvpoyvgfbzzgzxqzoeostjbffrzhppozgtqzflelwmaidfcebruhpwjfijmdaustwvzsghikdcggclservepthbjrvxoehfisjqqzqhvbkqrpycjxfgwdvoeqvealjwhivesescihveicycjozxsdvcdpiqgyhrvgzdsaoijelqrpvscsmqfoxbmccxsrhfvbtabdwflwjmbexbvvvxwnhojqvzrlqhvjvdgwdjfikqgaksiwhgsohgtwvglwdhvfiqbpoizafzhchelsuqgtozlgzabdhhusjroywojarscclsjafdowhgdozejzxgfarfvppokwhgsdfceqzesdsoizvzqsdwesdqbeuwxcldsfarvjfedcrdfgzfwzqgksemqpvskzrocyvhrbtqrpycjzfugohtvfmaidiozhvezflfvoljmpxlvpcaitviefvrzpwdlwjeoywrvqvfhpysiwkqgfsfvavpcywcefvyoctivzvydchwehvpoyvzffudsohzlbzhscvavftudzxfcsjescywtsjubyrasfrnzpvelsmaidpomsqdsygijavdqtsclfcqgbxocwkqgpwfrbxqfpvrfbkhcfvomsqqbclqywdabtqhvzcuupqqvgrzggrijrrzgxdzlhkqqzqhisctcxpsaoldotvdvikqhchsksmmwyfijoeejzxgzzdmicdwktruhcricsipoyvzvgrnzphhvasdodvsizrbcfvgzsiqrpvsjdzqrdvoegmaiddjvqlzsruwwtvbsciwuszxofuozhcmpzxfvarovllfvhdqgzvarwjvsxhglwjfsyxglfdqgrdfusjocxpslbrfvwhhvsobsclavbkqjzxgdsuabydhvgcmtcrwusldeflglfxuhohjfgtabnhdkwfzgdxpcwdqgpasddkqgohdrgjucymsdseescywjdfgfchxvhvdoghqusumwyosjxfgwdvoeqvesaksdsiqgohafbtaicwjfmrsspwdfiidsyycpsipsxddffkqzpvcwtiqgdbagoktwbxsjarugeucddvggpvrvavegpppcosxsdyclgdqrzqbrhvezlsflrvzqprdzbzmhchelcepsnkwwtiqonkohivbodgoegmagxhhycuqglgazfrnzpvrvzrzowbgvrvxodbbkvvespwzrrvpinwwfbaqapqgvfmugariirvdcfwsizveffvsjdvdbtfwvijqgohafbvzbppwdcifswsclfcmhedelsimazqhfiimjpfoufvegphhgzfzupurrbjxsdywjqvdsdgscvfyapxbgczsblurrwxgeflfvgkqfldxrarugpqtfbtqrlqgjceoccsgtoiosdwiessxsdviisuabelzesjqfposmsimdlvjfijysorbeokqgwdzfuzciptizsjfqzpavzryspozvavysohjfgvzgpluesdqbevdcszzgohgruvegpdjvqjqgdbzccxugxhgucefzpoosmiubekstcdbzttivbvzsdwelsgxidfcddiqvpqgzpcqazqwehvxztjseqvesywwkgroqcrwkfvpioriszvesdiciqveofgotwvggpvocozpsohqvhrgltowrwiqhpufzpcqxpgstclhftvrrbjxvfpoewkqsyqoxsrzhghfjzveplvtfbuesyiotsuqzpfivwcpswdvrweqzlpstvrzqpwseczdspwvzrvggptizqiaialgjozfofpwcwvgrpvazojysdgscskqfpvsegrpatuoehcqbzppiwcxsausdwvdxpgstclhftvrrbjxsdwsessdsdgsjsjqbeuozzcqgnhjzqvzsqdgkscqalogldvdwpxfvbcgwlxpzsemjpfqvhkqocpsvagawdrbesvcipyclgdqdchhrhvexpiwjrveqpqrisuqgzqdzsuqgedztceehcxwkdrdzlootvvfsohzycdyswhqisrfsfuzlwdqaplzxfzzqlgsjrvzhdhhjisuhnhhkszzxfuszueaatqwvijqqluwcommwesclfrpjpugrwiqefhzhilzrpszlgwafetivzluallgascqzllgjsimwohqfhvocxpslbgmefhhuswuqpozvgrrwygosozegpuafbmazwhdvbjqicgsjqrdhpvtrwjmwexbvtfugnhhksiqtwhlzcecipuwvbuqgzowusemjllhvhvnoelglfmaidfskozfiyharbzqfplbxseusfvsuswmwchqfagdsygfvelqzpsfvazqfghblbvbcfyozhjgfwhqfigpsnrimfzdjzwfvjrxsfuwesjfwxdpcsvzsqiskelawohdcijecwlrvelqzpvhiczeefdzzhvedclbtwgmzpvrvxrzcxpsvghgwdhzvjvzhpqhiscmqphgtcdysfqstcldcyqslbzcipviizvycxhbkolsidwsusmahchoiqyuhpfhlfvocwrgjocqazqidsefefluioepwevoegtqgdhrvrvocfysihvcizwwuwvzbpvrrbjhcdpwesjpsolodoefsegsodcaflwwfbjeqthbkwwuefhguoeejzvgldvdppvrfarubpvcdoktsxdhzelqgddwehvedflgjwvljzxggoihceustcdyscfsgsibsexscqfzgzosizvdsdwsusdqguriiguqzlpstvrzqpwsusctcxpsvhuqztqxlgkuqpgixfrzrerik"
tailleCle = rechercheTaillCle(txt)
print("Taille probable de la clé :" , tailleCle)

print(attaqueFreqVigenere(txt, tailleCle))

Taille probable de la clé : 8
8 mot(s) ont étés testés en 0.010804 seconde(s),
omathematiquesseveresjenevousaipasoublieesdepuisquevossavantesleconsplusdoucesquelemielfiltrerentdansmoncoeurcommeuneonderafraichissantejaspiraisinstinctivementdesleberceauaboireavotresourceplusanciennequelesoleiletjecontinueencoredefoulerleparvissacredevotretemplesolennelmoileplusfideledevosinitiesilyavaitduvaguedansmonespritunjenesaisquoiepaiscommedelafumeemaisjesusfranchirreligieusementlesdegresquimenentavotreauteletvousavezchassecevoileobscurcommeleventchasseledamiervousavezmisalaplaceunefroideurexcessiveuneprudenceconsommeeetunelogiqueimplacablealaidedevotrelaitfortifiantmonintelligencesestrapidementdeveloppeeetaprisdesproportionsimmensesaumilieudecetteclarteravissantedontvousfaitespresentavecprodigaliteaceuxquivousaimentdaunsincereamourarithmetiquealgebregeometrietrinitegrandiosetrianglelumineuxceluiquinevousapasconnuesestuninsenseilmeriteraitlepreuvedesplusgrandssuplicescarilyadumeprisaveugledanssonin

<div style="background-color:rgba(0, 255, 0, 0.19);padding:3%;">
<b>Réponse : </b>
Le livre s'intitule : Chants de Maldoror par le Compte de Lauréamont
</div>